# Replication Explanation 
This notebook aims to explain the definitions and data and how we manage to replicate the columns in the He table.

## Variable Descriptions
intermediary_capital_ratio
--------------------------
The end of period ratio of total market cap to (total market cap + book assets - book equity)
of NY Fed primary dealers' publicly-traded holding companies.

intermediary_capital_risk_factor
--------------------------------
AR(1) innovations to the intermediary_capital_ratio scaled by lagged intermediary_capital_ratio.
Note: In extending the intermediary_capital_risk_factor, we retain the AR(1)
coefficients used in the paper.

intermediary_value_weighted_investment_return
---------------------------------------------
The value-weighted investment return to a portfolio of NY Fed primary dealers'
publicly-traded holding companies. Unlike the intermediary_capital_risk_factor,
this portfolio is tradable, and performed similarly as a pricing factor.

intermediary_leverage_ratio_squared
-----------------------------------
This level variable, defined as (1/intermediary_capital_ratio)^2 was used
for preliminary predictive tests in the paper, as prescribed by the HK model.

## Pull NY Fed Data
(+variable codebook)

In [ ]:
#do i need to upload the file to github so that it can be downloaded automatically in the future?
import pandas as pd
import numpy as np
dfs_raw = pd.read_excel("../data_manual/Dealer_Lists_1960_to_2014.xls", sheet_name="Dealer Alpha",skiprows=2)

In [126]:
dealer_df = dfs_raw[dfs_raw["End Date"] == "Current Dealer"].drop(dfs_raw.columns[-1],axis=1)
dealer_df["Primary Dealer"] = dealer_df["Primary Dealer"].str.strip()
dealer_df.count()

Primary Dealer    22
Start Date        22
End Date          22
dtype: int64

### Manually add company name

In [127]:
comp_holding_company = ["(bank of nova scotia)","BARCLAYS PLC", "(bank of montreal)", "BNP PARIBAS", "(cantor firzgerald)", "CITIGROUP INC", "CREDIT SUISSE GROUP", "DAIWA SECURITIES GROUP INC", "DEUTSCHE BANK AG", "GOLDMAN SACHS GROUP INC", "HSBC HLDGS PLC", "JPMORGAN CHASE & CO", "JEFFERIES FINANCIAL GRP INC", "BANK OF AMERICA CORP", "MIZUHO FINANCIAL GROUP INC", "MORGAN STANLEY", "NOMURA HOLDINGS INC", "(rbc)", "ROYAL BANK OF SCOTLAND LTD", "SOCIETE GENERALE GROUP", "(td)", "UBS GROUP AG" ]

In [140]:
markit_holding_company = ["The Bank of Nova Scotia","Barclays PLC", "Bank of Montreal", "BNP PARIBAS", "Cantor Fitzgerald, L.P.", "Citigroup Inc.", "Credit Suisse Group AG", "Daiwa Securities Group Inc.", "DEUTSCHE BANK AKTIENGESELLSCHAFT", "The Goldman Sachs Group, Inc.", "HSBC HOLDINGS plc", "JPMorgan Chase & Co.", "Jefferies Group LLC", "Bank of America Corporation", "Mizuho Holdings Inc.", "Morgan Stanley", "Nomura Holdings, Inc.", "Royal Bank of Canada", "The Royal Bank Of Scotland Group public limited company", "SOCIETE GENERALE", "The Toronto-Dominion Bank", "UBS AG" ]

In [ ]:
dealer_df["markit"] = markit_holding_company
dealer_df["comp"] = comp_holding_company
dealer_df["comp"] = dealer_df["comp"].replace(r"\(.*\)", np.nan, regex=True)

In [ ]:
dealer_df = dealer_df.drop(columns=["End Date","Start Date"])
dealer_df = dealer_df.reset_index(drop=True)
dealer_df.to_csv("../data_manual/dealer_name_reference.csv")

In [146]:
dealer_df.head(5)

,Primary Dealer,markit,comp
0,"BANK OF NOVA SCOTIA, NEW YORK AGENCY",The Bank of Nova Scotia,NaN
1,BARCLAYS CAPITAL INC.,Barclays PLC,BARCLAYS PLC
2,BMO CAPITAL MARKETS CORP.,Bank of Montreal,NaN
3,BNP PARIBAS SECURITIES CORP.,BNP PARIBAS,BNP PARIBAS
4,CANTOR FITZGERALD & CO.,"Cantor Fitzgerald, L.P.",NaN


## Pull WRDS data 

https://wrds-www.wharton.upenn.edu/pages/support/manuals-and-overviews/compustat/north-america-global-bank/wrds-overview-compustat-north-america-global-bank/
GVKEY
DATADATE
INDFMT
DATAFMT
CONSOL
POPSRC

In [1]:
import wrds

# Connect to WRDS
from settings import config
# Make sure that you have included the line
# WRDS_USERNAME = your_wrds_username
# in the .env file
WRDS_USERNAME = config("WRDS_USERNAME") 
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# Check available datasets
db.list_libraries()[:10]

Loading library list...
Done


['aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank']

In [89]:
dfdf = db.get_table(library="markit",table="redent")

In [88]:
db.describe_table(library="markit",table="redent")

Approximately 17984 rows in markit.redent.


,name,nullable,type,comment
0,redcode,True,VARCHAR(6),REDCODE (Markit Primary Entity ID)
1,entity_cusip,True,VARCHAR(6),Ref Entity CUSIP (6 characters)
2,ticker,True,VARCHAR(30),Markit Entity Ticker
3,referenceentity,True,VARCHAR(500),Reference Entity Full Legal Name
4,shortname,True,VARCHAR(100),Entity Short Name
5,lei,True,VARCHAR(20),Legal Entity Identifier
6,entity_type,True,VARCHAR(20),Reference Entity Type
7,jurisdiction,True,VARCHAR(500),Jurisdiction of Entity
8,depthlevel,True,VARCHAR(4),Depth of Markit CDS Pricing
9,markitsector,True,VARCHAR(500),Markit Sectors Classification


In [144]:
merged_df = pd.merge(dfdf, dealer_df, left_on="referenceentity", right_on="markit", how="right")

In [148]:
merged_df.head(5)

,redcode,entity_cusip,ticker,referenceentity,shortname,lei,entity_type,jurisdiction,depthlevel,markitsector,entity_form,companynum_type,companynum,alternativenames,recorddate,validto,validfrom,Primary Dealer,markit,comp
0,CC6EHY,C0689P,BNS,The Bank of Nova Scotia,Bk Nova Scotia,L3I9ZG2KFGXZ61BMYR72,StatBody,Canada,None,Financials,CORP,None,None,La Banque de Nouvelle-Ecosse,2023-11-21,None,None,"BANK OF NOVA SCOTIA, NEW YORK AGENCY",The Bank of Nova Scotia,NaN
1,GG8839,G08036,BACR,Barclays PLC,Barclays PLC,213800LBQA1Y9L22JB70,Corp,England and Wales,High,Financials,PLC,English Company Number,48839,None,2025-02-10,None,None,BARCLAYS CAPITAL INC.,Barclays PLC,BARCLAYS PLC
2,CC676B,C06156,BMO,Bank of Montreal,Bk Montreal,NQQ6HPCNCCU6TUTQYE16,StatBody,Canada,None,Financials,CORP,None,None,Banque de Montreal,2022-05-19,None,None,BMO CAPITAL MARKETS CORP.,Bank of Montreal,NaN
3,05ABBF,05565A,BNP,BNP PARIBAS,BNP Paribas,R0MUWSFPU8MPRO8K5P83,Corp,France,High,Financials,SA,French Company Number (RCS Paris),662 042 449,None,2024-10-01,None,1998-05-12,BNP PARIBAS SECURITIES CORP.,BNP PARIBAS,BNP PARIBAS
4,14BE77,138616,CANFIT,"Cantor Fitzgerald, L.P.",Cantor Fitzgerald LP,549300RLUEEVQEMX0450,Corp,Delaware (US),None,Financials,None,None,None,None,None,None,None,CANTOR FITZGERALD & CO.,"Cantor Fitzgerald, L.P.",NaN


In [ ]:
## we need an extra step to delete the unnecessary BOA row, but unsure about what to delete yet.

## Merge with CRSP-Compustat
we followed this instruction: https://wrds-www.wharton.upenn.edu/pages/wrds-research/database-linking-matrix/linking-markit-with-crsp/

In [170]:

map_df = db.raw_sql(
'''
SELECT distinct
 b.permno, b.hdrCusip, b.ticker as crspTicker, b.permco, b.issuernm
FROM crsp.stksecurityinfohdr as b
''')

In [151]:
db.describe_table(library="crsp",table="stksecurityinfohdr")

Approximately 37796 rows in crsp.stksecurityinfohdr.


,name,nullable,type,comment
0,permno,True,INTEGER,PERMNO
1,secinfostartdt,True,DATE,Security Information Start Date
2,secinfoenddt,True,DATE,Security Information End Date
3,securitybegdt,True,DATE,Begin Date of Stock Data
4,securityenddt,True,DATE,End Date of Stock Data
5,securityhdrflg,True,VARCHAR(1),Security Header Flag
6,hdrcusip,True,VARCHAR(8),Header CUSIP -8 Characters
7,hdrcusip9,True,VARCHAR(9),Header CUSIP -9 Characters
8,cusip,True,VARCHAR(8),CUSIP
9,cusip9,True,VARCHAR(9),CUSIP9


In [152]:
db.describe_table(library="crsp",table="msf")

Approximately 5150989 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,PERMNO
2,permco,True,INTEGER,PERMCO
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code Header
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


In [175]:
### Get red entity information
redent = db.get_table(library='markit', table='redent')

# Quick check to confirm that it is the header information
# i.e. each redcode is mapped to only one entity 
# and doesn't contain historical records
redcnt = redent.groupby(['redcode'])['entity_cusip'].count().reset_index().rename(columns={'entity_cusip':'cusipCnt'})
redcnt.cusipCnt.max()
### Each redcode has only one entity_cusip mapped
### so it is indeed only header record

### Get information from CRSP header table
crspHdr = db.raw_sql('select permno, permco, hdrcusip, ticker, issuernm from crsp.stksecurityinfohdr')
crspHdr['cusip6'] = crspHdr.hdrcusip.str[:6]
crspHdr = crspHdr.rename(columns={'ticker': 'crspTicker'})

### First Route - Link with 6-digit cusip
_cdscrsp1 = pd.merge(redent, crspHdr, how='left', left_on='entity_cusip', right_on='cusip6')

# store linked results through CUSIP
_cdscrsp_cusip = _cdscrsp1.loc[_cdscrsp1.permno.notna()].copy()
_cdscrsp_cusip['flg'] = 'cusip'

# continue to work with non-linked records
_cdscrsp2 = _cdscrsp1.loc[_cdscrsp1.permno.isna()].copy().drop(columns=['permno','permco','hdrcusip','crspTicker','issuernm','cusip6'])

### Second Route - Link with Ticker
_cdscrsp3 = pd.merge(_cdscrsp2, crspHdr, how = 'left', left_on = 'ticker', right_on='crspTicker')
_cdscrsp_ticker = _cdscrsp3.loc[_cdscrsp3.permno.notna()].copy()
_cdscrsp_ticker['flg']='ticker'


In [176]:
import wrds
import os
import numpy as np
import pandas as pd

# display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import pickle as pkl
from fuzzywuzzy import fuzz

cdscrsp = pd.concat([_cdscrsp_cusip, _cdscrsp_ticker], ignore_index=True, axis=0)

# Check similarity ratio of company names
crspNameLst= cdscrsp.issuernm.str.upper().tolist()
redNameLst = cdscrsp.shortname.str.upper().tolist()
len(crspNameLst), len(redNameLst)

nameRatio  = [] # blank list to store fuzzy ratio

for i in range(len(redNameLst)):
    ratio = fuzz.partial_ratio(redNameLst[i], crspNameLst[i])
    nameRatio.append(ratio)
    
cdscrsp['nameRatio']=nameRatio

c:\miniforge3\envs\blank\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [189]:
final_df = cdscrsp.merge(merged_df["referenceentity"], on = "referenceentity", how="right")
final_df = final_df[final_df["redcode"].isna() == False]
final_df.drop_duplicates()

,redcode,entity_cusip,ticker,referenceentity,shortname,lei,entity_type,jurisdiction,depthlevel,markitsector,entity_form,companynum_type,companynum,alternativenames,recorddate,validto,validfrom,permno,permco,hdrcusip,crspTicker,issuernm,cusip6,flg,nameRatio
0,CC6EHY,C0689P,BNS,The Bank of Nova Scotia,Bk Nova Scotia,L3I9ZG2KFGXZ61BMYR72,StatBody,Canada,None,Financials,CORP,None,None,La Banque de Nouvelle-Ecosse,2023-11-21,None,None,42606.0,20339.0,05596130,BNS,B N S CO,055961,ticker,62.0
1,CC6EHY,C0689P,BNS,The Bank of Nova Scotia,Bk Nova Scotia,L3I9ZG2KFGXZ61BMYR72,StatBody,Canada,None,Financials,CORP,None,None,La Banque de Nouvelle-Ecosse,2023-11-21,None,None,89428.0,43264.0,06414910,BNS,BANK OF NOVA SCOTIA,064149,ticker,86.0
3,CC676B,C06156,BMO,Bank of Montreal,Bk Montreal,NQQ6HPCNCCU6TUTQYE16,StatBody,Canada,None,Financials,CORP,None,None,Banque de Montreal,2022-05-19,None,None,81284.0,29146.0,06367110,BMO,BANK MONTREAL QUE,063671,ticker,91.0
4,05ABBF,05565A,BNP,BNP PARIBAS,BNP Paribas,R0MUWSFPU8MPRO8K5P83,Corp,France,High,Financials,SA,French Company Number (RCS Paris),662 042 449,None,2024-10-01,None,1998-05-12,53487.0,25319.0,07011310,BNP,BASIN PETROLEUM CORP,070113,ticker,27.0
5,05ABBF,05565A,BNP,BNP PARIBAS,BNP Paribas,R0MUWSFPU8MPRO8K5P83,Corp,France,High,Financials,SA,French Company Number (RCS Paris),662 042 449,None,2024-10-01,None,1998-05-12,75037.0,20314.0,05564T10,BNP,B N P RESIDENTIAL PROPERTIES INC,05564T,ticker,55.0
7,189BFD,172967,C,Citigroup Inc.,Citigroup Inc,6SHGI4ZSSLCXXQSBB395,Corp,Delaware (US),High,Financials,CORP,None,None,None,2025-01-15,None,2005-08-01,70519.0,20483.0,17296742,C,CITIGROUP INC,172967,cusip,100.0
10,2H6677,251525,DB,DEUTSCHE BANK AKTIENGESELLSCHAFT,Deutsche Bk AG,7LTWFZYICNSX8D621K86,Corp,Germany,Med,Financials,AG,"Commercial Register Number, Frankfurt am Main,...",HRB 30000,None,2024-09-20,None,2002-08-08,89199.0,42291.0,D1819089,DB,DEUTSCHE BANK A G,D18190,ticker,86.0
11,3B955H,38141G,GS,"The Goldman Sachs Group, Inc.",Goldman Sachs Gp Inc,784F5XWPLTWKTBV3E584,Corp,Delaware (US),High,Financials,CORP,None,None,None,2024-08-21,None,None,86868.0,35048.0,38141G10,GS,GOLDMAN SACHS GROUP INC,38141G,cusip,85.0
12,4E46A8,404280,HSBC,HSBC HOLDINGS plc,HSBC Hldgs plc,MLU0ZO3ML4LN2LL2TL39,Corp,England and Wales,Med,Financials,PLC,English Company Number,617987,None,2025-01-16,None,None,87033.0,35175.0,40428040,HSBC,H S B C HOLDINGS PLC,404280,cusip,79.0
13,4C933G,48121F,JPM,JPMorgan Chase & Co.,JPMorgan Chase & Co,8I5DZWZKVSZI1NUHU748,Corp,Delaware (US),High,Financials,CORP,None,None,None,2025-01-16,None,2004-07-20,47896.0,20436.0,46625H10,JPM,JPMORGAN CHASE & CO,46625H,ticker,100.0


In [216]:
## manually select rows.
# Ensure all values in permno_lst are converted to integers
permno_lst = final_df["permno"].dropna().astype(int).tolist()

# Format as a properly structured SQL IN clause
permno_str = "(" + ",".join(map(str, permno_lst)) + ")"

# Print the formatted string (for debugging)
print(permno_str)  # Output should look like: (42606,89428,81284,53487,75037,...)


(42606,89428,81284,53487,75037,70519,89199,86868,87033,47896,48071,59408,59408,59408,69032,89256,82654,83835,15054,75849)


In [218]:
query = f"""
SELECT 
    date,
    msf.permno, msf.permco, shrcd, exchcd, comnam, shrcls, 
    ret, retx, dlret, dlretx, dlstcd,
    prc, altprc, vol, shrout, cfacshr, cfacpr,
    naics, siccd
FROM crsp.msf AS msf

LEFT JOIN 
    crsp.msenames as msenames
ON 
    msf.permno = msenames.permno AND
    msenames.namedt <= msf.date AND
    msf.date <= msenames.nameendt
LEFT JOIN 
    crsp.msedelist as msedelist
ON 
    msf.permno = msedelist.permno AND
    date_trunc('month', msf.date)::date =
    date_trunc('month', msedelist.dlstdt)::date
WHERE msf.permno IN {permno_str}
"""

db = wrds.Connection(wrds_username=WRDS_USERNAME)
df1 = db.raw_sql(query, date_cols=["date", "namedt", "nameendt", "dlstdt"])


Loading library list...
Done


In [221]:
df1.head(5)

,date,permno,permco,shrcd,exchcd,comnam,shrcls,ret,retx,dlret,dlretx,dlstcd,prc,altprc,vol,shrout,cfacshr,cfacpr,naics,siccd
0,2018-08-31,15054,55100,12.0,1.0,U B S GROUP A G,None,-0.046285,-0.046285,NaN,NaN,NaN,15.66,15.66,395926.0,3729120.0,1.0,1.0,525990,6021.0
1,2018-09-28,15054,55100,12.0,1.0,U B S GROUP A G,None,0.004470,0.004470,NaN,NaN,NaN,15.73,15.73,281879.0,3729120.0,1.0,1.0,525990,6021.0
2,2018-10-31,15054,55100,12.0,1.0,U B S GROUP A G,None,-0.116338,-0.116338,NaN,NaN,NaN,13.90,13.90,532175.0,3726373.0,1.0,1.0,525990,6021.0
3,2018-11-30,15054,55100,12.0,1.0,U B S GROUP A G,None,-0.026619,-0.026619,NaN,NaN,NaN,13.53,13.53,441854.0,3726373.0,1.0,1.0,525990,6021.0
4,2018-12-31,15054,55100,12.0,1.0,U B S GROUP A G,None,-0.084996,-0.084996,NaN,NaN,NaN,12.38,12.38,763803.0,3726373.0,1.0,1.0,525990,6021.0


In [226]:
len(df1.permno.value_counts())

18

In [223]:
df_counts = df1.groupby(["permno", "comnam"]).size()
print(df_counts)

permno  comnam                          
15054   U B S GROUP A G                     122
42606   B N S CO                             10
        BROWN & SHARPE MFG CO               425
47896   CHASE MANHATTAN CORP NEW             57
        CHEMICAL BANKING CORP                96
        CHEMICAL NEW YORK CORP              229
        J P MORGAN CHASE & CO                42
        JPMORGAN CHASE & CO                 246
48071   MORGAN J P & CO INC                 381
53487   BASIN PETROLEUM CORP                 50
59408   BANK OF AMERICA CORP                309
        BANKAMERICA CORP NEW                  6
        N C N B CORP                        229
        NATIONSBANK CORP                     81
69032   MORGAN STANLEY DEAN WITTER & CO     322
        MORGAN STANLEY DEAN WITTER D & C      9
        MORGAN STANLEY GROUP INC            135
70519   CITIGROUP INC                       315
        COMMERCIAL CREDIT CO                 19
        COMMERCIAL CREDIT GROUP INC           7

## Calculation

In [227]:
df1["adj_shrout"] = df1["shrout"] * df1["cfacshr"]
df1["adj_prc"] = df1["prc"].abs() / df1["cfacpr"]
df1["market_cap"] = df1["adj_prc"] * df1["adj_shrout"]

In [7]:
fundq_des = db.describe_table(library="comp",table="fundq")
fundq_des[fundq_des["name"]=="conm"]

Approximately 2048197 rows in comp.fundq.


,name,nullable,type,comment
11,conm,True,VARCHAR(70),Company Name


In [9]:
db = wrds.Connection(wrds_username=WRDS_USERNAME)

fundq_com_df = db.raw_sql(
    """
    SELECT DISTINCT conm
    FROM comp.fundq;
    """
)

Loading library list...
Done


In [13]:
fundq_com_df[fundq_com_df.conm.str.contains("ABN")]

,conm
7251,ABN-AMRO HOLDINGS NV
25659,YIELDMAX ABNB OPT INCM STRAT
